In [1]:
import numpy as np
import os,re,datetime
import pandas as pd
import pickle
from models import get_model
from tools.MyClock import MyClock
import scipy.sparse
import warnings
warnings.filterwarnings('ignore')
clk = MyClock()

Using TensorFlow backend.


# get predictions

In [2]:
MODEL_NAME = 'xmlcnn'
IN_DIR = 'data/dl_sic'
# IN_DIR = 'data/dl_amazon_1'
# MODEL_DIR = 'xmlcnn/amazon_model3.h5'
MODEL_DIR = 'xmlcnn/sic_model.h5'

In [3]:
in_dirs = {
    'embedding_matrix':'embedding_matrix.npy',
    'x_train':'x_train.npy',
    'x_test':'x_test.npy',
    'y_train':'y_train.npz',
    'y_test':'y_test.npz'
}
for key,val in in_dirs.items():
    d = os.path.join(IN_DIR,val)
    if not os.path.exists(d):
        raise Exception('path does not exist: {}'.format(d))
    else:
        in_dirs[key] = d
print('READ DATA...')
embedding_matrix = np.load(in_dirs['embedding_matrix'])
x_test = np.load(in_dirs['x_test'])
y_test = scipy.sparse.load_npz(in_dirs['y_test'])
y_test = y_test.todense()
labels_dim = y_test.shape[-1]
num_words,embedding_dim = embedding_matrix.shape
max_sequence_length = x_test.shape[1]

READ DATA...


In [4]:
model = get_model(MODEL_NAME,embedding_matrix,max_sequence_length,labels_dim)

W0715 21:22:59.847549 139828792592192 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0715 21:22:59.868210 139828792592192 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0715 21:23:00.147978 139828792592192 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0715 21:23:00.162779 139828792592192 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0715 21:23:00.220037 139828792

In [5]:
model.load_weights(MODEL_DIR)

W0715 21:23:00.307398 139828792592192 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [7]:
batch_size = x_test.shape[0]//100#1000
k=10
inds = []
probs = []
s = x_test.shape[0]
clk.tic()
for i,start in enumerate(range(0,s,batch_size)):
    end = min(start+batch_size,s)
    x_batch = x_test[start:end,:]
    out_probs = model.predict(x_batch)
    ind = np.argsort(out_probs,axis=1)[:,-k:]
    ind = ind[:,::-1]
    probs.append(np.take_along_axis(out_probs, ind, axis=1))
    inds.append(ind)
    print('{:0.0f}% {}'.format(end/s*100,clk.toc(False)),end='\r')

In [20]:
INDEX_DIR = 'xmlcnn/amazon3_ind.txt'
with open(INDEX_DIR,'ab') as f:
    for ind in inds:
        np.savetxt(f,ind,fmt='%d')
PROB_DIR = 'xmlcnn/amazon3_prob.txt'
with open(PROB_DIR,'ab') as f:
    for prob in probs:
        np.savetxt(f,prob,fmt='%1.3f')

# get p@5 (classification)

In [14]:
IN_DIR = 'data/dl_sic'
MODEL_DIRS = [
    'outputs/190722_204831_xmlcnn',
#     'outputs/190715_215817_attentionxml',
#     'outputs/190715_235041_attention',
]

y_test = scipy.sparse.load_npz(os.path.join(IN_DIR,'y_test.npz'))
y_test = np.argmax(y_test.todense(),axis=1).A1
k=5
for MODEL_DIR in MODEL_DIRS:
    IND_DIR = os.path.join(MODEL_DIR,'prediction_10_ind.txt')
    pred = np.loadtxt(IND_DIR)[:,:k].astype(int)
    correct = 0
    for i,y in enumerate(y_test):
        if y in pred[i,:]:
            correct+=1
    print('DIR: {}, p@{}: {:.4f}'.format(MODEL_DIR.split('/')[-1],k,correct/len(y_test)))

DIR: 190722_204831_xmlcnn, p@5: 0.7466
